# Step 4) Preparing training data

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from tsm.data_utils import time_processing, ordinal2wave, compress_memory_usage

/anaconda3/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
from tsm.normalizers import scale_and_save_data_frame_columns

In [5]:
import os

### Inputs

In [6]:
SEED = 42

In [7]:
np.random.seed(SEED)

## Pipeline for training data preparation

In [8]:
def pipeline_prepare_training_data(meter_no: int):
    data_path = 'data/comp/train_meter_{}.pkl'.format(str(meter_no))
    df = pd.read_pickle(data_path).reset_index(drop=True)
    df['log_meter_reading'] = np.log1p(df['meter_reading'])
    df = time_processing(df, 'timestamp')
    df = ordinal2wave('dt_m', df)
    df = ordinal2wave('dt_w', df)
    df = ordinal2wave('dt_d', df)
    df = ordinal2wave('dt_hour', df)
    df = ordinal2wave('dt_day_week', df)
    df = ordinal2wave('dt_day_month', df)
    df = ordinal2wave('dt_week_month', df)
    df['day_suspicious'] = df.groupby(["building_id","dt_d"])['meter_reading'].transform("min") < 1
    df['month_suspicious'] = df.groupby(["building_id","dt_m"])['meter_reading'].transform("mean") < 1
    df = scale_and_save_data_frame_columns(df=df, 
                                           columns=['dt_m', 'dt_w', 'dt_d', 'dt_hour', 'dt_day_week', 'dt_day_month', 'dt_week_month'],
                                           store_path='data/objects/train_meter_{}'.format(str(meter_no)))

    building_data = pd.read_pickle('data/prep/building_metadata.pkl')
    df = df.merge(building_data, on='building_id',  how='left')
    del building_data
    weather_data = pd.read_pickle('data/prep/weather_train.pkl')
    df = df.merge(weather_data, on=['site_id', 'timestamp'], how='left')
    del weather_data
    df.drop('meter_reading', axis=1, inplace=True)
    df, _ = compress_memory_usage(df)
    return df

In [16]:
%%time
for mtno in [1, 2, 3]:
    mt = pipeline_prepare_training_data(meter_no=mtno)
    joblib.dump(mt,'data/prep/train_meter_{}.pkl'.format(mtno))
    os.remove('data/comp/train_meter_{}.pkl'.format(mtno))

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int16 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  w


Memory usage pre-compression was 1372.1078491210938
Memory usage after-compression was 1037.0582580566406
This is  75.5813953488372% of the initial size


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int16 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  w


Memory usage pre-compression was 888.6311264038086
Memory usage after-compression was 671.6398048400879
This is  75.5813953488372% of the initial size


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int16 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by MinMaxScaler.
  w


Memory usage pre-compression was 414.6849899291992
Memory usage after-compression was 313.4247016906738
This is  75.5813953488372% of the initial size
CPU times: user 46.8 s, sys: 22.9 s, total: 1min 9s
Wall time: 1min 35s


In [15]:
joblib.dump(mt,'data/prep/train_meter_{}.pkl'.format(mtno))

['data/prep/train_meter_0.pkl']

In [14]:
from sklearn.externals import joblib